In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import time
import torch.nn.functional as F
# torch_geometric only used to load the Cora dataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
import torch_geometric.utils as U
import torch.optim as optim
import matplotlib.pyplot as plt
import torch_geometric.nn as geom_nn
from models import GATModule, ModifiedGATModule, GNNModule, MLPModule
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
%matplotlib inline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

datasetCora = Planetoid("/tmp/Cora", name="Cora")
datasetCiteSeer = Planetoid("/tmp/CiteSeer", name="CiteSeer")
datasetPubMed = Planetoid("/tmp/PubMed", name="PubMed")
datasetlist = [datasetCora, datasetCiteSeer, datasetPubMed]

In [3]:
class NodeClassification(nn.Module):

    def __init__(self, num_features, hidden_dim, gcn_layers, mlp_layers, num_classes, gnn_act_fn, mlp_act_fn, control_gate, dropout):
      super().__init__()

      self.GNN = None
      self.MLP = None
      
      # GNNModule()
      if control_gate == 'GATMLP':
        self.GNN = GATModule(num_features, hidden_dim, hidden_dim, gcn_layers, gnn_act_fn, dropout)
        self.MLP = MLPModule(hidden_dim, hidden_dim, num_classes, mlp_layers, mlp_act_fn, dropout)
      elif control_gate == 'ModifiedGATMLP':
        self.GNN = ModifiedGATModule(num_features, hidden_dim, hidden_dim, gcn_layers, gnn_act_fn, dropout)
        self.MLP = MLPModule(hidden_dim, hidden_dim, num_classes, mlp_layers, mlp_act_fn, dropout)
      elif control_gate == 'GCNMLP':
        self.GNN = GNNModule(num_features, hidden_dim, hidden_dim, gcn_layers, gnn_act_fn, dropout)
        self.MLP = MLPModule(hidden_dim, hidden_dim, num_classes, mlp_layers, mlp_act_fn, dropout)
      elif control_gate == 'ModifiedGAT':
        self.GNN = ModifiedGATModule(num_features, hidden_dim, num_classes, gcn_layers, gnn_act_fn, dropout)
      elif control_gate == 'GAT':
        self.GNN = GATModule(num_features, hidden_dim, num_classes, gcn_layers, gnn_act_fn, dropout)
      elif control_gate == 'GCN':
        self.GNN = GNNModule(num_features, hidden_dim, num_classes, gcn_layers, gnn_act_fn, dropout)
      elif control_gate == 'MLP':
        self.MLP = MLPModule(num_features, hidden_dim, num_classes, mlp_layers, mlp_act_fn, dropout)

    def forward(self, x, adj_matrix):
      if self.GNN is not None:
        x = self.GNN(x, adj_matrix)

      if self.MLP is not None:
        x = self.MLP(x)

      return x
  
    # Used to reset the weights of the network when training multiple times with
    # different hyperparameters
    def reset_parameters(self):
        def _reset_module_parameters(module):
            for layer in module.children():
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
                elif hasattr(layer, 'children'):
                    for child_layer in layer.children():
                        _reset_module_parameters(child_layer)

        _reset_module_parameters(self)

In [4]:
def train(model, data, adj_matrix, params):
    model.reset_parameters()
    model.train()
    # 定义优化器，通过优化器选择更新的方式
    opt =  optim.Adam(model.parameters(), lr = params["learning_rate"])
    # 定义损失函数，通过损失函数计算梯度
    loss_module = nn.CrossEntropyLoss()

    losses = []
    accuracies = []
    test_accuracies = []
    best_test = 0
    early_stopping = 0
    best_epoch = 0
    
    for e in range(0, params["num_epochs"]):
        # 清空之前的梯度
        opt.zero_grad()
        
        # 得到训练和测试向量
        train_d = data.train_mask
        test_d = data.test_mask

        train_pred = model(data.x, adj_matrix) # 分类预测
        # loss(input, target)
        loss = loss_module(train_pred[train_d], data.y[train_d])
        train_accuracy = (train_pred[train_d].argmax(dim=-1) == data.y[train_d]).sum().float() / train_d.sum()
        test_accuracy = (train_pred[test_d].argmax(dim=1) == data.y[test_d]).sum().float() / test_d.sum()

        #if e % 8 == 0:
        #    print(f"train_loss: {loss}, train_accuracy: {train_accuracy}, and test_accuracy: {test_accuracy}")
        losses.append(loss)
        accuracies.append(train_accuracy)
        test_accuracies.append(test_accuracy)
        if test_accuracy <= best_test:
            early_stopping += 1
        if best_test < test_accuracy:
            best_test = test_accuracy
            best_epoch = e
            early_stopping = 0
        if early_stopping == 10:
            print(f"Best test accuracy is {test_accuracy} with {best_epoch} epochs")
            break

        # 计算梯度
        loss.backward()
        # 更新参数
        opt.step()
    
    return losses, accuracies, test_accuracies, best_test, best_epoch

In [5]:
for model_name in ['ModifiedGAT', 'GAT', 'MLP', 'GCN']:
    print(f"This model is: {model_name}")
    with open('output.txt', 'w') as f:
        print(f"This model is: {model_name}", file = f)
    for dataset in datasetlist:
        num_nodes = dataset.data.num_nodes
        num_edges = dataset.data.num_edges // 2
        num_features = dataset.num_node_features
        num_classes = dataset.num_classes

        data = dataset[0].to(device)
        adj_matrix = U.to_dense_adj(data.edge_index).squeeze(0)

        params = {
            "hidden_features": 64,
            "num_gcn_layers": 2,
            "num_mlp_layers": 2,
            "learning_rate": 0.005,
            "weight_decay": 0,
            "num_epochs": 300,
            "dropout": 0.2,
        }

        model = NodeClassification(num_features,
                                    params["hidden_features"],
                                    params["num_gcn_layers"],
                                    params["num_mlp_layers"],
                                    num_classes,
                                    gnn_act_fn=nn.ReLU(inplace = True),
                                    mlp_act_fn=nn.ReLU(inplace = True),
                                    control_gate=model_name,
                                    dropout=params['dropout']
        ).to(device)
        timelength_list = []
        best_test_list = []
        for i in range(10):
            start = time.time()
            train_losses, train_accuracies, test_accuracies, best_test, best_epoch = train(model, data, adj_matrix, params)
            end = time.time()
            timelength = end - start

            timelength_list.append(timelength)
            best_test_list.append(best_test)

            #plt.plot([_ for _ in train_accuracies], label = "train_accuracy")
            #plt.plot([_ for _ in test_accuracies], label = "test_accuracy")
            #plt.legend()
            #plt.show()
        print(dataset.name + ": ")
        print(f"Mean best test accuracy: {np.mean(best_test_list)}; SD: {np.std(best_test_list)}")
        print(f"Mean runtime: {np.mean(timelength_list)}; Mean epoches: {best_epoch}")
        with open('output.txt', 'w') as f:
            print(dataset.name + ": ", file = f)
            print(f"Mean best test accuracy: {np.mean(best_test_list)}; SD: {np.std(best_test_list)}", file = f)
            print(f"Mean runtime: {np.mean(timelength_list)}; Mean epoches: {best_epoch}", file = f)

This model is: ModifiedGAT
Best test accuracy is 0.7789999842643738 with 6 epochs


KeyboardInterrupt: 